In [1]:
#Setup
import pandas as pd
import os
import pprint

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.2f}".format

<p style="font-family: Open Sans; color: #2B6264; font-weight: bold; font-size: 125%">What are we going to cover?</p>

<p style="font-family: Open Sans; color: #2B6264">- What are the available notification methods?</p>

<p style="font-family: Open Sans; color: #2B6264">- How do we use the Notifications API via the LUSID Website to surface data from an event in our email notifications via templating?</p>

<p style="font-family: Open Sans; color: #2B6264">- How can we use the Notifications API via the Python SDK to update our email notification to use HTML instead of plain text?</p>

<p style="font-family: Open Sans; color: #2B6264">- How can we use the Notifications API via the Python SDK to configure a text message notification?</p>

<p style="font-family: Open Sans; color: #2B6264">- How can we use the Notifications API via the Python SDK to delete a notification?</p>

<p style="font-family: Open Sans; color: #2B6264; font-weight: bold; font-size: 125%">What are the available notification methods?</p>

<p style="font-family: Open Sans; color: #2B6264">- Email</p>

<p style="font-family: Open Sans; color: #2B6264">- Text Message</p>

<p style="font-family: Open Sans; color: #2B6264">- Webhook</p>

<p style="font-family: Open Sans; color: #2B6264; font-weight: bold; font-size: 125%">Updating our email notification to use HTML</p>

In [ ]:
import lusid_notifications
from lusidjam import RefreshingToken
from lusid_notifications.extensions import (
    SyncApiClientFactory,
    ArgsConfigurationLoader,
    EnvironmentVariablesConfigurationLoader,
    SecretsFileConfigurationLoader
)
from lusid_notifications.models import *

# Authenticate to SDK
# Run the Notebook in Jupyterhub for your LUSID domain and authenticate automatically
secrets_path = os.getenv("FBN_SECRETS_PATH")
api_url_notifications = os.getenv("FBN_NOTIFICATIONS_API_URL")
# Run the Notebook locally using a secrets file (see https://support.lusid.com/docs/how-do-i-use-an-api-access-token-with-the-lusid-sdk)
if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
config_loaders=[
    ArgsConfigurationLoader(access_token = RefreshingToken(), app_name = "LusidJupyterNotebook", api_url=api_url_notifications),
    EnvironmentVariablesConfigurationLoader(),
    SecretsFileConfigurationLoader(secrets_path)]
api_factory = lusid_notifications.extensions.SyncApiClientFactory(config_loaders=config_loaders)

notifications_api = api_factory.build(lusid_notifications.api.NotificationsApi)

subscription_scope = "FinbourneUniversity"
subscription_code = "AnyPortfolioCreated"

In [3]:
subscriptions_api = api_factory.build(lusid_notifications.api.SubscriptionsApi)

try:
    subscriptions_api.get_subscription(
        scope=subscription_scope,
        code=subscription_code)
    
    subscriptions_api.delete_subscription(
        scope=subscription_scope,
        code=subscription_code
    )
    
except lusid_notifications.ApiException as api_exception:
    if api_exception.status == 404:
        pass
    else:
        raise

In [ ]:
subscriptions_api.create_subscription(
    create_subscription=lusid_notifications.CreateSubscription(
        id=lusid_notifications.ResourceId(
            scope=subscription_scope,
            code=subscription_code,
        ),
        display_name="Any Portfolio Created",
        description="Listens to all TransactionPortfolioCreated events",
        status="Active",
        matching_pattern=lusid_notifications.MatchingPattern(
            event_type="PortfolioCreated",
            filter=None
        )
    )
)

notifications_api.create_notification(
    scope = subscription_scope,
    code = subscription_code,
    create_notification_request = lusid_notifications.models.CreateNotificationRequest.from_dict(
        {
            "notificationId": "PortfolioCreatedEmail",
            "displayName": "PortfolioCreatedEmail",
            "description": "Portfolio created email notification",
            "notificationType": {
                "type": "Email",
                "subject": "New Portfolio Created!",
                "plainTextBody": """
                    A new portfolio called {{Body.portfolioCode}} has been created 
                    with an identifier of {{Body.portfolioScope}} / {{Body.portfolioCode}} 
                    and a creation date of {{Header.timestamp}}
                """,
                "HtmlBody": None,
                "emailAddressTo": ["jane.smith@acme.com"],
                "email_address_cc": [],
                "email_address_bcc": []
            }
        }
    )
)


In [ ]:
response = notifications_api.list_notifications(
    scope=subscription_scope,
    code=subscription_code)

pprint.pprint(response)

email_notification = response.values[0]

In [ ]:
with open("./data/EmailTemplate.html", 'r') as file:
    html_template = file.read()

print(html_template)

In [ ]:
updated_response = notifications_api.update_notification(
    scope = subscription_scope,
    code = subscription_code,
    id = "PortfolioCreatedEmail",
    update_notification_request = lusid_notifications.models.UpdateNotificationRequest.from_dict(
        {
            "displayName": "PortfolioCreatedEmail",
            "description": "Portfolio created email notification",
            "notificationType": {
                "type": "Email",
                "subject": "New Portfolio Created!",
                "plainTextBody": """
                    A new portfolio has been created 
                    with an identifier of {{Body.portfolioScope}} / {{Body.portfolioCode}} 
                    and a creation date of {{Header.timestamp}}.
                """,
                "HtmlBody": None,
                "emailAddressTo": ["jane.smith@acme.com"],
                "email_address_cc": ["joe.bloggs@acme.com"],
                "email_address_bcc": []
            }
        }
    )
)

pprint.pprint(updated_response)

<p style="font-family: Open Sans; color: #2B6264; font-weight: bold; font-size: 125%">Creating a text message notification</p>

In [ ]:
create_response = notifications_api.create_notification(
    scope = subscription_scope,
    code = subscription_code,
    create_notification_request = lusid_notifications.models.CreateNotificationRequest.from_dict(
        {
            "notificationId": "PortfolioCreatedSms",
            "displayName": "PortfolioCreatedSms",
            "description": "Portfolio created SMS notification",
            "notificationType": {
                "type": "Sms",
                "body": """A portfolio has been created 
                    wtih id {{Body.portfolioScope}}/{{Body.portfolioCode}}.
                    
                    Please ensure it has been configured correctly by visiting:
                    
                    https://demo.lusid.com/app/dashboard/holdings?scope={{Body.portfolioScope}}&code={{Body.portfolioCode}}&entityType=Portfolio""",
                "recipients": [
                    "+447700900000",
                    "+12334550000"
                ]
            }
        }
    )
)

pprint.pprint(create_response)

<p style="font-family: Open Sans; color: #2B6264; font-weight: bold; font-size: 125%">Deleting a notification</p>

In [ ]:
delete_response = notifications_api.delete_notification(
    scope=subscription_scope,
    code=subscription_code,
    id="PortfolioCreatedEmail"
)

pprint.pprint(delete_response)

In [15]:
subscriptions_api.delete_subscription(
    scope=subscription_scope,
    code=subscription_code
)


<p style="font-family: Open Sans; color: #2B6264; font-weight: bold; font-size: 125%">What have we covered?</p>

<p style="font-family: Open Sans; color: #2B6264">- There are three available notification methods. Email, text message and webhook.</p>

<p style="font-family: Open Sans; color: #2B6264">- You can use templating to surface data from the event in the notification. We set this up for an email notification using the Notifications API via the LUSID website.</p>

<p style="font-family: Open Sans; color: #2B6264">- You can also send an email in HTML, we set this up by updating our email notification using the Notifications API via the Python SDK.</p>

<p style="font-family: Open Sans; color: #2B6264">- You can also use templating for text message notifications. We created such a notification using the Notifications API via the Python SDK.</p>

<p style="font-family: Open Sans; color: #2B6264">- Notifications can be deleted from a subscription. We deleted the text message notification using the Notifications API via the Python SDK.</p>